# Neural Network

In [111]:
import numpy as np
import pickle
import gzip


def load_data():
    """
    MNIST comprises of three variables ->
    Training Data:     50,000 entries
    Validation Data:   10,000 entries
    Test Data:         10,000 entries

    One Entry of Input = 28 * 28 = 784
    One Entry of Output = 0 - 9 {integer}

    training_data   =  (inputs, outputs)     - Tuple
    inputs.shape    =  (50,000, 784)         - Numpy Array
    outputs.shape   =  (50,000,)             - Numpy Array
    """
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(
        f, encoding='iso-8859-1')
    f.close()
    return (training_data, validation_data, test_data)


def load_data_wrapper():
    """
    It Returns Training Data, Validation Data, Test Data

    Training Data is a list of 50,000 entries.
    Each element of this list is a tuple in the form (Input, Output)
    Shape of 1 Input : (784,1)
    Shape of 1 Output : (10,1)

    Validation Data is a list of 10,000 entries.
    Each element of this list is a tuple in the form (Input, Output)
    Shape of 1 Input : (784,1)
    Shape of 1 Output : Integer

    Test Data is a list of 10,000 entries.
    Each element of this list is a tuple in the form (Input, Output)
    Shape of 1 Input : (784,1)
    Shape of 1 Output : Integer
    """
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)


def vectorized_result(j):
    """
    Return a 10 bit vectorized result of a number.
    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [112]:
import numpy as np
import random

In [113]:
### Mathematical Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [118]:
class Network:
    
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        """Returns the output value."""
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, a) + b
            a = sigmoid(z)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data = None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batches = []
            for k in range(0, n, mini_batch_size):
                mini_batch = training_data[k : k + mini_batch_size]
                X, Y = zip(*mini_batch)
                X = np.array(X)
                Y = np.array(Y)
                X = X.reshape(X.shape[0], X.shape[1]).transpose()
                Y = Y.reshape(Y.shape[0], Y.shape[1]).transpose()
                mini_batch = (X, Y)
                mini_batches.append(mini_batch)
                
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print('Epoch {0}: {1} / {2}'.
                     format(epoch, self.evaluate(test_data), n_test))
            else:
                print('Epoch {0} complete.'.format(epoch))
    
    def update_mini_batch(self, mini_batch, eta):
        x, y = mini_batch
        delta_b, delta_w = self.backprop(x, y)
            
        m = mini_batch[0].shape[1]
        self.biases = [b - (eta/m) * db
                       for b, db in zip(self.biases, delta_b)]
        self.weights = [w - (eta/m) * dw
                        for w, dw in zip(self.weights, delta_w)]
    
                      
    def backprop(self, x, y):
        delta_b = [np.zeros(b.shape) for b in self.biases]
        delta_w = [np.zeros(w.shape) for w in self.weights]
        
        # forward pass
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward pass
        delta = (activations[-1] - y) * sigmoid_prime(zs[-1])
        delta_b[-1] = np.sum(delta, axis = 1).reshape(delta.shape[0], 1)
        delta_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sigmoid_prime(z)
            delta_b[-l] = np.sum(delta, axis = 1).reshape(delta.shape[0], 1)
            delta_w[-l] = np.dot(delta, activations[-l-1].transpose())
            
        return (delta_b, delta_w)
        
    def evaluate(self, test_data):
        result = 0
        for x, y in test_data:
            y_hat = np.argmax(self.feedforward(x))
            if (y_hat == y):
                result += 1
        return result        

In [124]:
net = Network([784, 30, 10])

In [120]:
training_data, validation_data, test_data = load_data_wrapper()

In [125]:
net.SGD(training_data, 30, 30, 3.0, test_data = test_data)

Epoch 0: 8029 / 10000
Epoch 1: 8226 / 10000
Epoch 2: 9053 / 10000
Epoch 3: 9149 / 10000
Epoch 4: 9225 / 10000
Epoch 5: 9281 / 10000
Epoch 6: 9287 / 10000
Epoch 7: 9324 / 10000
Epoch 8: 9308 / 10000
Epoch 9: 9325 / 10000
Epoch 10: 9333 / 10000
Epoch 11: 9358 / 10000
Epoch 12: 9347 / 10000
Epoch 13: 9354 / 10000
Epoch 14: 9375 / 10000
Epoch 15: 9369 / 10000
Epoch 16: 9375 / 10000
Epoch 17: 9361 / 10000
Epoch 18: 9381 / 10000
Epoch 19: 9390 / 10000
Epoch 20: 9385 / 10000
Epoch 21: 9396 / 10000
Epoch 22: 9389 / 10000
Epoch 23: 9393 / 10000
Epoch 24: 9397 / 10000
Epoch 25: 9396 / 10000
Epoch 26: 9406 / 10000
Epoch 27: 9405 / 10000
Epoch 28: 9401 / 10000
Epoch 29: 9402 / 10000
